In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
driver = webdriver.Chrome("/Users/alliewu/Desktop/22 Summer/Projects/Web Scraping/chromedriver")

/var/folders/y2/gl6v6t_n63g4cqcss8z26cj00000gn/T/ipykernel_30318/1344583770.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/Users/alliewu/Desktop/22 Summer/Projects/Web Scraping/chromedriver")


In [3]:
url = 'https://www.indeed.com/jobs?q=data%20analyst&l=United%20States&sc=0kf%3Aexplvl(ENTRY_LEVEL)%3B&vjk=fb4561a71d5af956'
driver.get(url)
driver.implicitly_wait(3)

In [7]:
job = []

for i in range(1,5):
    all_jobs = driver.find_elements(By.CLASS_NAME, 'resultContent')

    for jobs in all_jobs:
        html = jobs.get_attribute('innerHTML')
        soup = BeautifulSoup(html, 'html.parser')

        title = soup.find(class_ = "jobTitle").text
        company = soup.find(class_ = "companyName").text
        location = soup.find(class_ = "companyLocation").text
        try:
            salary = soup.find_all(class_ = "attribute_snippet")[0].text
        except:
            salary = None
        #post_date = soup.find('span',class_ = 'date').text.replace("Posted","")
        href = soup.find(href = True)['href'].replace(';',"&")
        #href = href.replace('/company/','/viewjob?cmp=')
        href = href.replace('/rc/clk?', '/viewjob?')
        dic = {'title': title, 'company': company, 'location': location, 'salary': salary, 'href': href}
        job.append(dic)
    
    driver.find_element(By.XPATH, '//*[@id="resultsCol"]/nav/div/ul/li[6]/a/span').click()

In [10]:
webs = []
more = []

for i in range(0,len(job['href'])):
    url = str('https://www.indeed.com'+ job['href'][i])
    webs.append(url)
    driver.get(url)
    driver.implicitly_wait(3)
    
    try:
        benefits = driver.find_element(By.XPATH, '//*[@id="benefits"]/div/div').text.replace('\n', ' ')
    except:
        benefits = None
        
    try:
        descriptions = driver.find_element(By.XPATH, '//*[@id="jobDescriptionText"]').text.replace('\n', ' ')
    except:
        descriptions = None 
        
    try:
        post_date = driver.find_element(By.XPATH, '//*[@id="hiringInsightsSectionRoot"]/p[3]').text
    except:
        post_date = None
    
    dic2 = {'benefits': benefits, 'description': descriptions, 'date': post_date}
    more.append(dic2)

In [40]:
job = pd.DataFrame(job)
more = pd.DataFrame(more)
website = pd.DataFrame(webs)
website.columns = ['website']

result = pd.concat([job, more, website], axis = 1)
result = result.drop(labels = 'href', axis = 1)
result = result[['date','company','title','location','salary','benefits','description','website']]
result.head()

,date,company,title,location,salary,benefits,description,website
0,Employer reviewed job 18 days ago,TECH AVEEV LLC,Data Analyst,Remote,Part-time +2,None,Develop data solutions to support business nee...,https://www.indeed.com/company/TECHAVEEV/jobs/...
1,Posted 21 days ago,NuWest Group,Data Analyst,"San Diego, CA","$3,000 a week",None,Data Analyst needed at two sites in San Diego ...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
2,None,Mayo Clinic,Data Science Analyst- Remote,"Jacksonville, FL 32224 (Beach Haven area)+1 lo...","$89,773 - $134,638 a year",None,Why Mayo Clinic Mayo Clinic is the nation's be...,https://www.indeed.com/viewjob?jk=4e6f400d7098...
3,None,Michael Page,Remote Data Manipulation Analyst,"Remote in Bensalem, PA","$100,000 - $120,000 a year",Paid time off,You gain access to databases of acquired syste...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
4,Posted 30+ days ago,Insight Global,Data Analyst,"San Francisco Bay Area, CA+1 location",$50 - $60 an hour,401(k) Dental insurance Health insurance Visio...,We are looking for a Data Analyst to work for ...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...


In [41]:
from datetime import date
today = date.today()
filename = f'Indeed_DataAnalyst_Job_{today}'
result.to_excel('{}.xlsx'.format(filename), index=False)